In [3]:
import theano
import theano.tensor as T
import numpy as np
import cv2
import sys
from loader import PatchLoader

In [4]:
from easydict import EasyDict
cfg = EasyDict()
cfg.SEQ_LENGTH = 250
cfg.TILE_SIZE = 9
cfg.OUT_SIZE = 1
cfg.gm_num = 4
cfg.ndim = 4
cfg.NAME_PREFIX = 'nt'
cfg.NETWORK = 'conv4_net_dense'
cfg.NAME='experiments/'+cfg.NAME_PREFIX+cfg.NETWORK+'%d'%(cfg.ndim)

cfg.TRAIN = EasyDict()
cfg.TRAIN.EPOCH = 0
cfg.TRAIN.TRIN_EPOCH_SIZE = 1500
cfg.TRAIN.TEST_EPOCH_SIZE = 750
cfg.TRAIN.EPOCH_NUM = 20

cfg.DATASET = EasyDict()
cfg.DATASET.TRAIN_MINR = 0.2
cfg.DATASET.TRAIN_MAXR = 0.3
cfg.DATASET.TEST_MINR = 0.15
cfg.DATASET.TEST_MAXR = 0.2
cfg.DATASET.T_SIZE = 32
cfg.DATASET.CASHE_SAMPLES = True

In [5]:

train_loader = PatchLoader('data/train_%d'%cfg.DATASET.T_SIZE,
                           cfg.DATASET.T_SIZE,
                           cfg.SEQ_LENGTH,
                           cfg.DATASET.TRAIN_MINR,
                           cfg.DATASET.TRAIN_MAXR,
                           cfg.TILE_SIZE,
                           cfg.DATASET.CASHE_SAMPLES)

In [6]:
# from dataset_tools import draw
# from loader import data_generator
# import matplotlib.pyplot as plt
# %matplotlib inline
# for x,y in data_generator(train_loader,epoch_size=1,shuffle=False):
#     print x.shape,y.shape
#     print np.ones_like(y)[y<0.1].sum(),np.ones_like(y)[y>0.9].sum()
#     x = np.transpose(x,(0,2,3,1)).astype(np.uint8)
#     y = (y*255.).astype(np.uint8)
#     _y = np.zeros(x.shape[:3],dtype=np.uint8)
    
#     _y[:] = y[:,0,0][:,None,None]
#     draw(x,_y)

# Build model

# Train

In [7]:
from gmm_op import get_gmm,calc_log_prob_gmm
from theano_utils import split,histogram_loss,split_tr_p_n
import lasagne
from networks import make_FCN


def make_classifier(X,label,non_learn_params):
    X = X.reshape((-1,X.shape[-1]))
    x_tr,x_p,x_n = split_tr_p_n(X,label.flatten())
    m,c,w = get_gmm(x_tr,
                    gm_num = non_learn_params['gm_num'],
                    ndims = non_learn_params['ndim'],
                    use_approx_grad=non_learn_params['use_approx_grad'])
    p_n = calc_log_prob_gmm(x_n,m,c,w)
    p_p = calc_log_prob_gmm(x_p,m,c,w)
    loss = histogram_loss(T.tanh(p_n),T.tanh(p_p),
                          non_learn_params['min_cov'],
                          non_learn_params['histogram_bins'],
                          non_learn_params['width'])[0]
    prediction = T.nnet.sigmoid(T.concatenate([p_p,p_n],axis=0))
    Y = T.concatenate([T.ones_like(p_p),T.zeros_like(p_n)],axis=0)
    return loss,X,Y,prediction,m,c,w,p_p,p_n

def make_train(net,data,label,non_learn_params):
    sym = L.get_output(net ,deterministic=False)
    s = int((L.get_output_shape(net)[1]-1)/2)
    sym = sym[:,s:s+1,s:s+1,:]
    loss,X,Y,prediction,m,c,w,p_p,p_n = make_classifier(sym,label[:,s:s+1,s:s+1],non_learn_params)
    params = L.get_all_params(net,trainable=True)
    grads = T.grad(loss,params)
    if( 'total_grad_constraint' in non_learn_params.keys()):
        grads = lasagne.updates.total_norm_constraint(grads,non_learn_params['total_grad_constraint'])
    updates = lasagne.updates.momentum(grads,params,
                                       learning_rate=non_learn_params['lr'],
                                       momentum=non_learn_params['momentum'])
    return theano.function([data, label], [loss,X,Y,prediction,m,c,w,p_n,p_p],\
                               allow_input_downcast=True, updates=updates)

def iterate_batches(fn,data_generator,epoch,metrix = dict(),logger=None):
    loss=0
    acc=0
    labels,predicted = np.array([]),np.array([])
    for i,batch in enumerate(data_generator()):
        if(batch[1][:,4,4][(batch[1][:,4,4]<0.9)&(batch[1][:,4,4]>0.1)].size > len(batch[0])//4):
            continue
            print ("skip")
        res = fn(*batch)
        mask = (res[2]>0.9) | (res[2]<0.1)
        loss+=res[0]
        labels = np.concatenate((labels,res[2][mask]))
        predicted = np.concatenate((predicted,res[3][mask]))
    s = ' '.join(['%s=%.3f'%(k,metrix[k](labels,predicted)) for k in metrix.keys() ])
    logger.log('epoch %i batch %i loss=%.2f l=%.2f %s\n'%(epoch,i,loss/float(i+1),res[0],s))

In [8]:
from sklearn.metrics import average_precision_score
from sklearn import metrics
import datetime
import os
from test_tools import make_test,GMMAlgorithm,bin_score
from utils import Logger
import lasagne.layers as L
from loader import data_generator


def update_params(epoch,params):
    if(epoch == 4):
        params['min_cov'].set_value(1e-8)
        params['lr'].set_value(0.00001)          
    if(epoch == 10):
        params['min_cov'].set_value(1e-8)
        params['lr'].set_value(0.000001)
    if(epoch == 15):
        params['lr'].set_value(0.0000001)
        
try:
    os.mkdir(cfg.NAME)
except Exception as e:
    print 'cannot create dir %s'%(cfg.NAME)
    print e

logger =Logger('std',cfg.NAME+"/train.log")

logger.log('################### train network '+cfg.NAME+ ' ' + str(datetime.datetime.now())+'################\n')    
logger.log(str(cfg))

try:
    os.mkdir('%s/models'%(cfg.NAME))
except Exception as e:
    logger.log('cannot create dir %s/models'%(cfg.NAME))
    print e
    
metrix = {'aps' : average_precision_score,
              'f1' : lambda y,s : metrics.f1_score(y,bin_score(s)),
              'acc' : lambda y,s : metrics.accuracy_score(y,bin_score(s))}

non_learn_params={'min_cov' : theano.shared(1e-3),
                  'lr' : theano.shared(np.array(1e-2, dtype=theano.config.floatX)),
                  'width': theano.shared(4.),
                  'total_grad_constraint': 10,
                  'histogram_bins' : 100,
                  'use_approx_grad' : True,
                  'ndim' : cfg.ndim,
                  'gm_num' : cfg.gm_num,
                  'momentum' : theano.shared(np.array(0.9, dtype=theano.config.floatX))}

cannot create dir experiments/ntconv4_net_dense4
[Errno 17] File exists: 'experiments/ntconv4_net_dense4'
################### train network experiments/ntconv4_net_dense4 2018-05-02 20:07:38.495173################

{'ndim': 4, 'NETWORK': 'conv4_net_dense', 'SEQ_LENGTH': 250, 'DATASET': {'TEST_MINR': 0.15, 'CASHE_SAMPLES': True, 'TRAIN_MAXR': 0.3, 'TRAIN_MINR': 0.2, 'T_SIZE': 32, 'TEST_MAXR': 0.2}, 'TRAIN': {'EPOCH': 0, 'EPOCH_NUM': 20, 'TRIN_EPOCH_SIZE': 1500, 'TEST_EPOCH_SIZE': 750}, 'gm_num': 4, 'OUT_SIZE': 1, 'TILE_SIZE': 9, 'NAME_PREFIX': 'nt', 'NAME': 'experiments/ntconv4_net_dense4'}
cannot create dir experiments/ntconv4_net_dense4/models
[Errno 17] File exists: 'experiments/ntconv4_net_dense4/models'


In [9]:
data = T.tensor4(name='data')
label = T.tensor3(name='label')
net = make_FCN(cfg.NETWORK,data,
               ndim=cfg.ndim,
               model_name='%s/models/%03d'%(cfg.NAME,cfg.TRAIN.EPOCH) if cfg.TRAIN.EPOCH > 0 else '',
               input_shape = (None,3,cfg.TILE_SIZE,cfg.TILE_SIZE),
               pad = 'same',
               logger=logger)
train_fn = make_train(net,data,label,non_learn_params)
logger.log('train_fn compiled')

load finction conv4_net_dense

In -->     Layer    --> Out    Description                
-------    -----    -------    -----------                
[]         0        [1]        data(None, 3, 9, 9)        
[0]        1        [2]        conv_1(None, 6, 9, 9)      
[1]        2        [3]        bn_1(None, 6, 9, 9)        
[2]        3        [4]        relu_1(None, 6, 9, 9)      
[3]        4        [5]        conv_2(None, 12, 9, 9)     
[4]        5        [6]        bn_2(None, 12, 9, 9)       
[5]        6        [7]        relu_2(None, 12, 9, 9)     
[6]        7        [8]        conv_3(None, 24, 9, 9)     
[7]        8        [9]        bn_3(None, 24, 9, 9)       
[8]        9        [10]       relu_3(None, 24, 9, 9)     
[9]        10       [11]       conv_4(None, 3, 9, 9)      
[10]       11       [12]       transpose(None, 9, 9, 3)   
[11]       12       [13]       l2norm(None, 9, 9, 4)      
[12]       13       []         normed_dense(None, 9, 9, 4)
train_fn compiled


In [10]:
from multiprocessing import Pool

globalPool = Pool(4)

In [13]:
non_learn_params['lr'].set_value(0.001)
#non_learn_params['beta1'].set_value(0.0)

In [15]:
from utils import save_weights

def gen_FCN(network,model,ndim):
    def res(data,input_shape):
        net = make_FCN(network,data,
                       ndim=ndim,
                       model_name=model,
                       input_shape = input_shape,
                       pad = 'same')
        return L.get_output(net)
    return res

for j in range(cfg.TRAIN.EPOCH):
    update_params(j,non_learn_params)

# logger.log('test')
# test_algorithm = GMMAlgorithm(gen_FCN(cfg.NETWORK,'',cfg.ndim),
#                                cfg.gm_num,
#                                globalPool)
# result = make_test(test_algorithm,
#       out_dir=None,
#       dataset='data/test',
#       train_size = 100,
#       test_size=100,
#       im_size = (240//2,320//2),
#       metrics = metrix,
#       logger=logger,
#       only_with_motion = True)

for j in range(cfg.TRAIN.EPOCH,cfg.TRAIN.EPOCH_NUM):
    update_params(j,non_learn_params)
    logger.log('train')
    iterate_batches(train_fn,\
                    lambda : data_generator(train_loader,epoch_size=10000000,shuffle=True),
                   j,metrix,logger = logger)
    save_weights(net,'%s/models/%03d'%(cfg.NAME,j))
    if((j+1)%20 == 0):
        logger.log('test')
        test_algorithm = GMMAlgorithm(gen_FCN(cfg.NETWORK,'%s/models/%03d'%(cfg.NAME,j),cfg.ndim),
                                       cfg.gm_num,
                                       globalPool)
        result = make_test(test_algorithm,
              out_dir=None,
              dataset='data/test',
              train_size = 100,
              test_size=100,
              im_size = (240//2,320//2),
              metrics = metrix,
              logger=logger,
              only_with_motion = True)


logger.log('################### done #######################\n')


train
epoch 0 batch 1186 loss=896682.87 l=14072894.83 acc=0.766 aps=0.762 f1=0.842

checkpoint experiments/ntconv4_net_dense4/models/000.npz
train


KeyboardInterrupt: 

In [ ]:
for i,batch in enumerate(data_generator(test_loader,epoch_size=100,shuffle=False)):
    res = test_fn(*batch)
    if(i == 0):
        p =res[-1]
        n = res[-2]
    p = np.concatenate((p,res[-1]))
    n = np.concatenate((n,res[-2]))
plt.hist(np.clip(res[-1],-10,100),100,normed=True,label='positive')
plt.hist(np.clip(res[-2],-10,100),100,normed=True,label='negative')
plt.legend()
plt.show()  
plt.hist(np.clip(p,-20,100),100,normed=True,label='positive')
plt.hist(np.clip(n,-20,100),100,normed=True,label='negative')
plt.legend()
plt.show()